In [ ]:
import math
import pandas as pd
import numpy as np
import pgljupyter
import openalea.plantgl.all as pgl
from vmapplet.tools.surface import leafSurface

device manual: https://polhemus.com/_assets/img/FASTRAK_User_Manual_OPM00PI002-G.pdf

In [ ]:
branches = pd.read_csv('branches.csv')
leaves = pd.read_csv('leaves.csv')

In [ ]:
branches

In [ ]:
leaves

In [ ]:
branch_shapes = []
for i, branch in branches.iterrows():
    if not i % 2:
        x0 = branch['X']
        y0 = branch['Y']
        z0 = branch['Z']
    else:
        x1 = branch['X']
        y1 = branch['Y']
        z1 = branch['Z']
        geom = pgl.Extrusion(pgl.Polyline([(x0, y0, -z0), (x1, y1, -z1)]), pgl.Polyline2D.Circle(1))
        shape = pgl.Shape(geom, pgl.Material(ambient=(49,29,25) , diffuse = 2.55102, shininess=0))
        branch_shapes.append(shape)

In [ ]:
leaf_shapes = []
for _, leaf in leaves.iterrows():
    x = leaf['X']
    y = leaf['Y']
    z = leaf['Z']
    a = leaf['A'] * math.pi / 180
    c = leaf['B'] * math.pi / 180
    b = leaf['C'] * math.pi / 180
    geom = pgl.Translated(x, y, -z, pgl.EulerRotated(a, b, c, leafSurface()))
    shape = pgl.Shape(geom, pgl.Material(ambient=(0, 59, 6), shininess=0, diffuse=1.59322))
    leaf_shapes.append(shape)

In [ ]:
pgljupyter.SceneWidget(branch_shapes + leaf_shapes, scale=1, size_world=-branches['Z'].min())

In [ ]:
def get_leaf_geom():
    import math
    points = [(0,-1,0), (0,1,0), (4,1,0), (4,-1,0)]
    indices = [(0, 1, 2, 3)]
    material = pgl.Material(ambient=(0, 59, 6), shininess=0, diffuse=1.59322)
    geom = pgl.QuadSet(points,indices)
    return geom
    #return pgl.EulerRotated(0, math.pi/2, 0,  pgl.Translated(0, 0, 1.5, pgl.Group([geom, pgl.Translated(0, 0, -1.5, pgl.Cylinder(0.05, 2.5))])))
    #return pgl.Group([geom, pgl.Translated(0, 0, -1.5, pgl.Cylinder(0.05, 2.5))])

In [ ]:
def plot_single_branch(code: str, widget: pgljupyter.SceneWidget):
    """branch bein and leaves translated to orgin 0,0,0"""
    branch = branches[branches['code rameau'] == code]
    leaves_ = leaves[leaves['code rameau'] == code]
    shapes = []

    x0 = branch.iloc[0]['X']
    y0 = branch.iloc[0]['Y']
    z0 = branch.iloc[0]['Z']
    x1 = branch.iloc[1]['X'] - x0  # remove offset to plot branch at origin (0,0,0)
    y1 = branch.iloc[1]['Y'] - y0
    z1 = branch.iloc[1]['Z'] - z0
    
    geom = pgl.Extrusion(pgl.Polyline([(0, 0, 0), (x1, y1, -z1)]), pgl.Polyline2D.Circle(1))
    shape = pgl.Shape(geom, pgl.Material(ambient=(49,29,25) , diffuse = 2.55102, shininess=0))
    shapes.append(shape)

    for _, leaf in leaves_.iterrows():
        x = leaf['X'] - x0
        y = leaf['Y'] - y0
        z = leaf['Z'] - z0
        a = leaf['A'] * math.pi / 180
        b = leaf['B'] * math.pi / 180
        c = leaf['C'] * math.pi / 180
        #  leafSurface()
        geom =  get_leaf_geom()
        geom = pgl.Translated(x, y, -z, pgl.EulerRotated(a, b, c, get_leaf_geom()))
        shape = pgl.Shape(geom, pgl.Material(ambient=(0, 59, 6), shininess=0, diffuse=1.59322))
        shapes.append(shape)

    widget.set_scenes(pgl.Scene(shapes), scales=0.1)

In [ ]:
single_branch_widget = pgljupyter.SceneWidget()
single_branch_widget

In [ ]:
plot_single_branch('F36PL2', single_branch_widget)

In [ ]:
leaf_widget = pgljupyter.SceneWidget()
leaf_widget

In [ ]:
leaf = pgl.Shape(get_leaf_geom(), pgl.Material(ambient=(0, 59, 6), shininess=0, diffuse=1.59322))
leaf_widget.set_scenes(leaf, scales=0.1)